*Worked done by Xingyue Huang and Ruotong Cao, under supervision of Dr.Cris Salvi. for 2022 Summer project in DataSig*

This is the script that is used to generate MSA feature, .pssm file, and corrosponding protein-related feature for training of a protein-folding problem. 

Input of the protein is an **.ent file** / **.pdb** file, e.g. T1019s2.pdb,
stored in dir ./ProteinPDB/AlphaProtein/, or any other dir, replaced 

*In addition, the input protein pdb will first be "fixed", and in replace of the original pdb file.*

Output file will be stored in dir ./test_data/ . Only the **.seq** file which stores the amino-sequence and final output **.npy** file which stores the calculated results for MSA will be under ./test_data/. All the intermediate file will be stored in new directory ./test_data/\{Target\}/ 

For each protein, we do the following generation in sequence:
 - Use pdbfixer to add missing residues if needed
 - Extract the amino-sequence from **.ent** file and produce **.seq**  file
 - Crop the sequence using window size 64,128,256

  For each of the cropped version + original:
  - Generate MSA profile using hhblits. Output the **.hhm**, **.hhr**, and **.a3m** file
  - Convert the **.a3m** file to **.fas** file
  - Using PSI-BLAST to take in **.fas** file and generate PSSM (position-specific scoring matrix) and stored in **.pssm** file
  - Generate feature vector and produce **.aln** file
  - Using PlmDCA to update **.aln** file (Not used)

- Read in each cropped's file and produce the final version of **.npy** file



# Set up the enviroment

Prerequisite:

1. Need to download and unzip the git repo for hhsuite at https://github.com/soedinglab/hh-suite  and store under /Feature_Generation/

2. Need to download and unzip the git repo for plmDCA at https://github.com/magnusekeberg/plmDCA , But we didn't use this yet. Store under /Feature_Generation 

3. For alpha-protein data, we download it form https://www.rcsb.org/stats/explore/polymer_entity_type, select **SCOP_Classification**, select **Alpha and beta proteins (a/b)**. In the left bar, tick **Protein** in tag **POLYMER ENTITY TYPE**, and click download button on the top-right. Copy all the 4-letter code and stored it into the file **alphaprotein_list.txt** .In the *cde-protein-folding.ipynb*, there are code on top to automatically download all the proteins. 

WARNING: It might take hours to download all the file, and even more with the MSA calculation
	

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
'''
import os
#PATH = "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/Feature_generation"
PATH = "/content/drive/MyDrive/Colab Notebook/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/Feature_generation/"
os.chdir(PATH)
'''

'\nimport os\n#PATH = "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/Feature_generation"\nPATH = "/content/drive/MyDrive/Colab Notebook/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/Feature_generation/"\nos.chdir(PATH)\n'

In [3]:
'''
!git clone https://github.com/soedinglab/hh-suite.git
!mkdir -p hh-suite/build && cd hh-suite/build
!cmake -DCMAKE_INSTALL_PREFIX=. ..
!make -j 4 && make install
!export PATH="$(pwd)/bin:$(pwd)/scripts:$PATH"

# Download Databases
!wget --no-check-certificate http://wwwuser.gwdg.de/~compbiol/uniclust/2018_08/uniclust30_2018_08_hhsuite.tar.gz 

'''

'\n!git clone https://github.com/soedinglab/hh-suite.git\n!mkdir -p hh-suite/build && cd hh-suite/build\n!cmake -DCMAKE_INSTALL_PREFIX=. ..\n!make -j 4 && make install\n!export PATH="$(pwd)/bin:$(pwd)/scripts:$PATH"\n\n# Download Databases\n!wget --no-check-certificate http://wwwuser.gwdg.de/~compbiol/uniclust/2018_08/uniclust30_2018_08_hhsuite.tar.gz \n\n'

In [4]:
# Unzip the databases for MSA: this requires a lot of space about 70G.
# WARNING： if the hhblits stops prematurely with no output, it is possibly because the unzip process of databases has gone wrong, and hhblits will stop without throwing any error.
# To solve this, delete the old databases file and unzip again, with enough memory. 
'''
%cd "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/Feature_generation"
!tar xzvf uniclust30_2018_08_hhsuite.tar.gz
'''

'\n%cd "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/Feature_generation"\n!tar xzvf uniclust30_2018_08_hhsuite.tar.gz\n'

In [5]:
# Set-up plmDCA file
'''
%cd /content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/Feature_generation
!git clone https://github.com/magnusekeberg/plmDCA.git
%mv plmDCA/plmDCA_asymmetric_v2 plmDCA/plmDCA
%cp ../plmDCA.m plmDCA/plmDCA_asymmetric_v2/

!apt-get install octave
!apt-get install liboctave-dev
# mex .c file, if you use matlab you need do this in matlab console
%cd plmDCA/plmDCA_asymmetric_v2/functions/
!for i in *.c; do octave --eval "mex $i";done
%cd ../3rd_party_code/minFunc/ 
!for i in *.c; do octave --eval "mex $i"; done
'''

'\n%cd /content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/Feature_generation\n!git clone https://github.com/magnusekeberg/plmDCA.git\n%mv plmDCA/plmDCA_asymmetric_v2 plmDCA/plmDCA\n%cp ../plmDCA.m plmDCA/plmDCA_asymmetric_v2/\n\n!apt-get install octave\n!apt-get install liboctave-dev\n# mex .c file, if you use matlab you need do this in matlab console\n%cd plmDCA/plmDCA_asymmetric_v2/functions/\n!for i in *.c; do octave --eval "mex $i";done\n%cd ../3rd_party_code/minFunc/ \n!for i in *.c; do octave --eval "mex $i"; done\n'

Set up pdbfixer for fixing missing file

In [6]:
# Consider using pdbfixer! 
'''
!pip install condacolab
import condacolab
condacolab.install()

!conda install -c conda-forge pdbfixer 
!conda install -c conda-forge openmm
'''

'\n!pip install condacolab\nimport condacolab\ncondacolab.install()\n\n!conda install -c conda-forge pdbfixer \n!conda install -c conda-forge openmm\n'

# Generating individual feature

In [7]:
'''
%cd "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc"
'''

'\n%cd "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc"\n'

In [8]:
# Set up the test target
'''
TARGET="T1019s2"
TARGET_DIR="test_data"
TARGET_SEQ= f"{TARGET_DIR}/{TARGET}.seq" # fasta format
PLMDCA_DIR="Feature_generation/plmDCA/plmDCA_asymmetric_v2/"
'''

'\nTARGET="T1019s2"\nTARGET_DIR="test_data"\nTARGET_SEQ= f"{TARGET_DIR}/{TARGET}.seq" # fasta format\nPLMDCA_DIR="Feature_generation/plmDCA/plmDCA_asymmetric_v2/"\n'

In [9]:
# generate domain crops from target seq
'''
!python3 feature.py -s $TARGET_SEQ -c
'''

'\n!python3 feature.py -s $TARGET_SEQ -c\n'

In [10]:
'''
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge -c bioconda hhsuite 
'''

'\n!pip install -q condacolab\nimport condacolab\ncondacolab.install()\n!conda install -c conda-forge -c bioconda hhsuite \n'

In [11]:
'''
!apt-get install ncbi-blast+
'''

'\n!apt-get install ncbi-blast+\n'

In [12]:

# Generate MSA file for all the seq. under
'''
import glob
%cd "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/test_data"
for file in glob.glob("*.seq"):
  out = file[:-4]
  print(f"Generate MSA files for {out}.seq")
  !hhblits -i {out}.seq -d ../Feature_generation/uniclust30_2018_08/uniclust30_2018_08 -oa3m {out}.a3m -ohhm {out}.hhm -n 3 
  !perl ../Feature_generation/hh-suite/scripts/reformat.pl {out}.a3m {out}.fas
  !psiblast -subject {out}.seq -in_msa {out}.fas -out_ascii_pssm {out}.pssm;
'''

'\nimport glob\n%cd "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/test_data"\nfor file in glob.glob("*.seq"):\n  out = file[:-4]\n  print(f"Generate MSA files for {out}.seq")\n  !hhblits -i {out}.seq -d ../Feature_generation/uniclust30_2018_08/uniclust30_2018_08 -oa3m {out}.a3m -ohhm {out}.hhm -n 3 \n  !perl ../Feature_generation/hh-suite/scripts/reformat.pl {out}.a3m {out}.fas\n  !psiblast -subject {out}.seq -in_msa {out}.fas -out_ascii_pssm {out}.pssm;\n'

In [13]:
'''
!pip install torch
!pip install numpy
!pip install scipy
'''

'\n!pip install torch\n!pip install numpy\n!pip install scipy\n'

In [14]:
# make target features data and generate ungap target aln file for plmDCA
'''
%cd /content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc
!python3 feature.py -s {TARGET_SEQ} -f
'''

'\n%cd /content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc\n!python3 feature.py -s {TARGET_SEQ} -f\n'

In [15]:
# Calculate plmDCA file to generate the alignment
'''
%cd ./test_data
for aln in glob.glob("*.aln"):
  out = file[:-4]
  print(f"calculate plmDCA for {aln}")
  !octave "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/Feature_generation/plmDCA/plmDCA_asymmetric_v2/plmDCA_asymmetric.m" {aln}

'''

'\n%cd ./test_data\nfor aln in glob.glob("*.aln"):\n  out = file[:-4]\n  print(f"calculate plmDCA for {aln}")\n  !octave "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/Feature_generation/plmDCA/plmDCA_asymmetric_v2/plmDCA_asymmetric.m" {aln}\n\n'

In [16]:
# run again to update target features data
'''
%cd ../
!python3 feature.py -s {TARGET_SEQ} -f
'''

'\n%cd ../\n!python3 feature.py -s {TARGET_SEQ} -f\n'

In [17]:
# Check the dimension of the feature output
'''
import numpy as np
dataset = np.load("./test_data/T1019s2.npy",allow_pickle = True)
item = dataset[0]
for key in item.keys():
  print(key, np.shape(item[key]))
'''

'\nimport numpy as np\ndataset = np.load("./test_data/T1019s2.npy",allow_pickle = True)\nitem = dataset[0]\nfor key in item.keys():\n  print(key, np.shape(item[key]))\n'

There might be the case where the PDB file has missing atom. We called **pdbfixer**, which can automatically fill the missing residue as well

In [18]:
'''
PATH = "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/ProteinPDB/AlphaProtein/"
pdbfile = "pdb6sii.ent"
outfile = "pdb6sii.ent"
target = pdbfile[:-3]
'''

'\nPATH = "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/ProteinPDB/AlphaProtein/"\npdbfile = "pdb6sii.ent"\noutfile = "pdb6sii.ent"\ntarget = pdbfile[:-3]\n'

In [19]:
# Consider using pdbfixer! 
'''
!pip install condacolab
import condacolab
condacolab.install()

!conda install -c conda-forge pdbfixer 
!conda install -c conda-forge openmm


%cd {PATH}
!pdbfixer {pdbfile} --add-residues --output={outfile} 
'''

'\n!pip install condacolab\nimport condacolab\ncondacolab.install()\n\n!conda install -c conda-forge pdbfixer \n!conda install -c conda-forge openmm\n\n\n%cd {PATH}\n!pdbfixer {pdbfile} --add-residues --output={outfile} \n'

# Batch-geneneration as a function

Now we summarise the above and generate the following function:

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import os
PATH = "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/Feature_generation"
#os.chdir("/content/drive/MyDrive/Colab Notebook/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/Feature_generation/")
os.chdir(PATH)

In [22]:
# pre-requisite
!pip install torch
!pip install scipy
!apt-get install octave
!apt-get install liboctave-dev
!apt-get install ncbi-blast+
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge -c bioconda hhsuite 
!conda install -c conda-forge pdbfixer 
!conda install -c conda-forge openmm
!pip install Biopython


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aglfn epstool fonts-droid-fallback fonts-noto-mono ghostscript gnuplot-data
  gnuplot-qt gsfonts imagemagick-6-common info install-info libamd2
  libauthen-sasl-perl libcamd2 libccolamd2 libcholmod3 libcolamd2
  libcupsfilters1 libcupsimage2 libcxsparse3 libdata-dump-perl libemf1
  libencode-locale-perl libfftw3-single3 libfile-listing-perl libfltk-gl1.3
  libfltk1.3 libfont-afm-perl libgail-common libgail18 libglpk40
  libgraphicsmagick++-q16-12 libgraphicsmagick-q16-3 libgs9 libgs9-common
  libgtk2.0-0 libgtk

In [23]:
from Bio import SeqIO
# PRE: target.pdb/.ent file, stored in ProteinPDB/AlphaProtein
# OUT: target.seq file, stored inside test_data
def pdb2seq(target,IN_PATH = "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/ProteinPDB/AlphaProtein"):
  try:
    pdb_file = IN_PATH + "/" + target+".ent"
    with open(pdb_file) as handle:
      sequence = next(SeqIO.parse(handle, "pdb-atom"))
  except:
    pdb_file = IN_PATH + "/" + target+".pdb"
    with open(pdb_file) as handle:
      sequence = next(SeqIO.parse(handle, "pdb-atom"))
  OUT_PATH = "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/test_data"
  with open(OUT_PATH + "/" + target+".seq", "w") as output_handle:
      SeqIO.write(sequence, output_handle, "fasta")

In [24]:
# We combine all the substance and write down the following function
# PRE: target.seq file stored inside test_data
# OUT: target.npy file with list of dict of cropped seq and their features, stored inside test_data
import glob
def generate_feature(target):
  # note target is in form T1019s2
  %cd /content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/test_data
  TARGET= target
  !mkdir {TARGET}
  !cp {TARGET}.seq {TARGET}/{TARGET}.seq
  %cd ../
  TARGET_DIR=f"test_data/{TARGET}"
  TARGET_SEQ= f"{TARGET_DIR}/{TARGET}.seq" # fasta format
  PLMDCA_DIR="Feature_generation/plmDCA/plmDCA_asymmetric_v2/"
  # generate domain crops from target seq
  !python3 feature.py -s {TARGET_SEQ} -c
  %cd ./{TARGET_DIR}
  for file in glob.glob("*.seq"):
    out = file[:-4]
    print(f"Generate MSA files for {out}.seq")
    !hhblits -i {out}.seq -d ../../Feature_generation/uniclust30_2018_08/uniclust30_2018_08 -oa3m {out}.a3m -ohhm {out}.hhm -n 3 
    !perl ../../Feature_generation/hh-suite/scripts/reformat.pl {out}.a3m {out}.fas
    !psiblast -subject {out}.seq -in_msa {out}.fas -out_ascii_pssm {out}.pssm;
  # make target features data and generate ungap target aln file for plmDCA
  %cd /content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc
  !python3 feature.py -s {TARGET_SEQ} -f
  # Calculating plmDCA
  %cd ./{TARGET_DIR}
  for aln in glob.glob("*.aln"):
    out = file[:-4]
    print(f"calculate plmDCA for {aln}")
    !octave "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/Feature_generation/plmDCA/plmDCA_asymmetric_v2/plmDCA_asymmetric.m" {aln}
  %cd ../../
  # run again to update target features data
  !python3 feature.py -s {TARGET_SEQ} -f
  %cd {TARGET_DIR}
  !cp {TARGET}.npy ../{TARGET}.npy

In [25]:
#generate_feature("T1019s2")

In [26]:
%cd /content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc

/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc


In [27]:

# test for IO error
import numpy as np
np = np.load("./test_data/T1019s2.npy",allow_pickle = True)
print(np[0].keys())

dict_keys(['chain_name', 'domain_name', 'sequence', 'seq_length', 'residue_index', 'aatype', 'hhblits_profile', 'reweighted_profile', 'hmm_profile', 'num_alignments', 'deletion_probability', 'gap_matrix', 'non_gapped_profile', 'pseudo_frob', 'pseudo_bias', 'pseudolikelihood', 'num_effective_alignments', 'mutual_information', 'resolution', 'sec_structure', 'sec_structure_mask', 'solv_surf', 'solv_surf_mask', 'alpha_positions', 'alpha_mask', 'beta_positions', 'beta_mask', 'superfamily', 'between_segment_residues', 'phi_angles', 'phi_mask', 'psi_angles', 'psi_mask', 'profile', 'profile_with_prior', 'profile_with_prior_without_gaps'])


In [ ]:
%cd ProteinPDB/AlphaProtein/
for i, item in enumerate(glob.glob("*.ent")):
  if i > 2:
    continue
  pdb_target = item[:-4]
  pdbfile = item
  outfile = pdbfile
  !pdbfixer {pdbfile} --add-residues --output={outfile} 
  pdb2seq(pdb_target)
  generate_feature(pdb_target)


/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/ProteinPDB/AlphaProtein


/usr/local/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 18144.
  PDBConstructionWarning,
/usr/local/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 18154.
  PDBConstructionWarning,
/usr/local/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 18208.
  PDBConstructionWarning,
/usr/local/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 18281.
  PDBConstructionWarning,
/usr/local/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 18339.
  PDBConstructionWarning,
/usr/local/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 18424.
  PDBC

/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/test_data
mkdir: cannot create directory ‘pdb6smy’: File exists
/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc
Traceback (most recent call last):
  File "feature.py", line 1, in <module>
    import utils
  File "/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/utils.py", line 2, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'
/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/test_data/pdb6smy
Generate MSA files for pdb6smy.seq
- 18:35:53.892 INFO: Search results will be written to pdb6smy.hhr

- 18:36:06.145 INFO: Searching 15161831 column state sequences.

- 18:36:06.356 INFO: pdb6smy.seq is in A2M, A3M or FASTA format

- 18:36:06.357 INFO: Neutra

/usr/local/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 9164.
  PDBConstructionWarning,
/usr/local/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 9208.
  PDBConstructionWarning,
/usr/local/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 9252.
  PDBConstructionWarning,
/usr/local/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 9296.
  PDBConstructionWarning,
/usr/local/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 9340.
  PDBConstructionWarning,
/usr/local/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 9557.
  PDBConstru

/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/test_data
mkdir: cannot create directory ‘pdb6smz’: File exists
/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc
python3: can't open file 'feature.py': [Errno 2] No such file or directory
/content/drive/MyDrive/Colab Notebooks/alphafold_pytorch/alphafold_pytorch-29bd20b0ca0ccd31d984a7dc53f0302d945a2ddc/test_data/pdb6smz
Generate MSA files for pdb6smz.seq
- 18:52:31.926 INFO: Search results will be written to pdb6smz.hhr

- 18:52:42.543 INFO: Searching 15161831 column state sequences.

- 18:52:42.624 INFO: pdb6smz.seq is in A2M, A3M or FASTA format

- 18:52:42.625 INFO: Iteration 1

- 18:52:42.947 INFO: Prefiltering database

- 18:53:40.439 INFO: HMMs passed 1st prefilter (gapless profile-profile alignment)  : 296618

- 18:53:44.153 INFO: HMMs passed 2nd prefilter (gapped profile-profile alignment)   : 3653